In [ ]:
# fill in the path in your Google Drive in the string below. Note: do not escape slashes or spaces
import os
datadir = "path/"
if not os.path.exists(datadir):
  !ln -s "path/" $datadir
os.chdir(datadir)
!pwd

In [ ]:
# downloading Fashion-MNIST
import os
os.chdir(os.path.join(datadir,"fashion-mnist/"))
!chmod +x ./get_data.sh
!./get_data.sh
os.chdir(datadir)

# Imports

In [ ]:
import random
import numpy as np
import pandas as pd
from data_process import get_FASHION_data, get_RICE_data
from scipy.spatial import distance
from models import Perceptron, SVM, Softmax, Logistic
from kaggle_submission import output_submission_csv
%matplotlib inline

# For auto-reloading external modules
# See http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Loading Fashion-MNIST

As part of the preprocessing pipeline, I defined the dataset splits. The combined training and validation set included 60,000 images, while the test set consisted of 10,000 images. These were then loaded and prepared for training and evaluation.

In [ ]:
# You can change these numbers for experimentation
# For submission we will use the default values
TRAIN_IMAGES = 50000
VAL_IMAGES = 10000
normalize = False

In [ ]:
data = get_FASHION_data(TRAIN_IMAGES, VAL_IMAGES, normalize=normalize)
X_train_fashion, y_train_fashion = data['X_train'], data['y_train']
X_val_fashion, y_val_fashion = data['X_val'], data['y_val']
X_test_fashion, y_test_fashion = data['X_test'], data['y_test']
n_class_fashion = len(np.unique(y_test_fashion))

# Standardizing the input features
mean_fmnist = np.mean(X_train_fashion, axis=0)
std_fmnist = np.std(X_train_fashion, axis=0)

X_train_fashion = (X_train_fashion - mean_fmnist) / (std_fmnist + 1e-8)  # Avoid division by zero
X_val_fashion = (X_val_fashion - mean_fmnist) / (std_fmnist + 1e-8)
X_test_fashion = (X_test_fashion - mean_fmnist) / (std_fmnist + 1e-8)

# Adding the bias term
X_train_fashion = np.concatenate((np.ones((X_train_fashion.shape[0], 1)), X_train_fashion), axis=1)
X_val_fashion = np.concatenate((np.ones((X_val_fashion.shape[0], 1)), X_val_fashion), axis=1)
X_test_fashion = np.concatenate((np.ones((X_test_fashion.shape[0], 1)), X_test_fashion), axis=1)

In [ ]:
# Checking bias term
X_train_fashion_df = pd.DataFrame(X_train_fashion)
X_train_fashion_df.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,1.0,-0.00892,-0.022843,-0.038668,-0.042039,-0.05897,-0.071120,-0.098286,-0.156807,-0.239129,...,-0.600456,-0.474466,-0.393991,-0.404943,-0.441078,-0.396479,-0.287818,-0.156079,-0.089607,-0.033796
1,1.0,-0.00892,-0.022843,-0.038668,-0.042039,-0.05897,0.102318,-0.098286,-0.156807,-0.239129,...,1.471715,1.860906,2.706860,1.332602,-0.441078,-0.396479,-0.287818,-0.156079,-0.089607,-0.033796
2,1.0,-0.00892,-0.022843,-0.038668,-0.042039,-0.05897,-0.071120,-0.098286,-0.156807,-0.239129,...,-0.600456,-0.474466,-0.370139,-0.404943,-0.441078,-0.396479,-0.287818,-0.156079,-0.089607,-0.033796
3,1.0,-0.00892,-0.022843,-0.038668,-0.042039,-0.05897,-0.071120,-0.098286,-0.156807,1.140259,...,-0.600456,-0.474466,-0.393991,-0.404943,-0.441078,-0.396479,-0.287818,-0.156079,-0.089607,-0.033796
4,1.0,-0.00892,-0.022843,-0.038668,-0.042039,-0.05897,-0.071120,-0.098286,-0.156807,-0.239129,...,-0.600456,-0.474466,-0.393991,-0.404943,-0.441078,-0.396479,-0.287818,-0.156079,-0.089607,-0.033796


In [ ]:
X_train_fashion.shape

(50000, 785)

In [ ]:
# Check labels
np.unique(y_train_fashion)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

# Loading Rice

In [ ]:
# loads train / test / val splits of 80%, 20%, 20%
data = get_RICE_data()
X_train_RICE, y_train_RICE = data['X_train'], data['y_train']
X_val_RICE, y_val_RICE = data['X_val'], data['y_val']
X_test_RICE, y_test_RICE = data['X_test'], data['y_test']
n_class_RICE = len(np.unique(y_test_RICE))

# Normalizing the input features
X_min = np.min(X_train_RICE, axis=0)
X_max = np.max(X_train_RICE, axis=0)

X_train_RICE = (X_train_RICE - X_min) / (X_max - X_min + 1e-8)  # Avoid division by zero
X_val_RICE = (X_val_RICE - X_min) / (X_max - X_min + 1e-8)
X_test_RICE = (X_test_RICE - X_min) / (X_max - X_min + 1e-8)

# Adding bias term
X_train_RICE = np.concatenate((np.ones((X_train_RICE.shape[0], 1)), X_train_RICE), axis=1)
X_val_RICE = np.concatenate((np.ones((X_val_RICE.shape[0], 1)), X_val_RICE), axis=1)
X_test_RICE = np.concatenate((np.ones((X_test_RICE.shape[0], 1)), X_test_RICE), axis=1)

print("Number of train samples: ", X_train_RICE.shape[0])
print("Number of val samples: ", X_val_RICE.shape[0])
print("Number of test samples: ", X_test_RICE.shape[0])

Number of train samples:  10911
Number of val samples:  3637
Number of test samples:  3637


In [ ]:
# Checking bias term
X_train_RICE_df = pd.DataFrame(X_train_RICE)
X_train_RICE_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,0.356436,0.407258,0.687030,0.306649,0.920318,0.397153,0.491233,0.392875,0.427973,0.606272,0.653406
1,1.0,0.858306,0.643730,0.672747,0.641852,0.757312,0.622030,0.712551,0.469772,0.487210,0.794097,0.353259
2,1.0,0.767272,0.775234,0.742581,0.748991,0.737778,0.765718,0.823969,0.402064,0.567474,0.780315,0.331277
3,1.0,0.122002,0.296436,0.516953,0.266393,0.878944,0.293812,0.375204,0.336607,0.331205,0.635043,0.547932
4,1.0,0.699230,0.929240,0.821544,0.861049,0.722044,0.911510,0.946311,0.595672,0.636390,0.803058,0.314802


In [ ]:
X_train_RICE.shape

(10911, 12)

In [ ]:
# Check labels
np.unique(y_test_RICE)

array([-1,  1])

### Get Accuracy

In [ ]:
def get_acc(pred, y_test):
    return np.sum(y_test == pred) / len(y_test) * 100

# Perceptron

In this project, I implemented a Perceptron classifier and explored the effects of two key hyperparameters: learning rate and number of training epochs. Through experimentation and tuning, I was able to observe how these parameters influenced model performance and convergence behavior.

The implementation was developed in **models/perceptron.py**.

### Hyperparameter 1: Learning rate
The learning rate determines the magnitude of weight updates during training. I began with a default value of 0.5 and experimented with a range of values from 0.0005 to 5.0 to study their impact on accuracy and training dynamics.

Here’s what I found:
* High learning rates (e.g., 0.5 or above) sometimes caused the model to diverge or exhibit unstable loss patterns. Reducing the rate by a factor of 10 (e.g., from 0.5 to 0.05) helped stabilize training.
* Low learning rates led to very slow convergence. In such cases, increasing the rate (e.g., from 0.005 to 0.05) improved training speed.
* I also experimented with learning rate decay by reducing the learning rate gradually over epochs (e.g., multiplying by 0.95 after each epoch), which helped fine-tune the model as training progressed.
* To visualize the effect of different learning rates, I plotted training and validation accuracy across epochs.

### Hyperparameter 2: Number of Epochs
An epoch refers to one full pass through the training dataset, where the model makes predictions and updates weights according to the Perceptron update rule.

To evaluate convergence, I:
* Monitored accuracy trends on both the training and validation sets.
* Increased the number of epochs when I noticed that accuracy plateaued early or the model failed to converge.
* Compared performance across different values of epochs to find an optimal balance between training time and accuracy.

These hyperparameter experiments provided valuable insights into the sensitivity of the Perceptron model and helped me better understand how to tune simple neural classifiers for improved performance.

## Train Perceptron on Fashion-MNIST

In [ ]:
lr = 3.5
n_epochs = 40
n_features_fashion = X_train_fashion.shape[1]

percept_fashion = Perceptron(n_features_fashion, n_class_fashion, lr, n_epochs)

# - Refer to hints in the train method of the Perceptron class in "models/perceptron.py"
percept_fashion.train(X_train_fashion, y_train_fashion)

Epoch 1/40, Training accuracy: 11.70%, Lr: 3.5000
Epoch 2/40, Training accuracy: 45.65%, Lr: 3.3250
Epoch 3/40, Training accuracy: 51.98%, Lr: 3.1587
Epoch 4/40, Training accuracy: 52.33%, Lr: 3.0008
Epoch 5/40, Training accuracy: 54.80%, Lr: 2.8508
Epoch 6/40, Training accuracy: 69.77%, Lr: 2.7082
Epoch 7/40, Training accuracy: 71.55%, Lr: 2.5728
Epoch 8/40, Training accuracy: 71.16%, Lr: 2.4442
Epoch 9/40, Training accuracy: 74.50%, Lr: 2.3220
Epoch 10/40, Training accuracy: 75.21%, Lr: 2.2059
Epoch 11/40, Training accuracy: 75.95%, Lr: 2.0956
Epoch 12/40, Training accuracy: 75.89%, Lr: 1.9908
Epoch 13/40, Training accuracy: 76.91%, Lr: 1.8913
Epoch 14/40, Training accuracy: 75.28%, Lr: 1.7967
Epoch 15/40, Training accuracy: 76.41%, Lr: 1.7069
Epoch 16/40, Training accuracy: 77.81%, Lr: 1.6215
Epoch 17/40, Training accuracy: 78.52%, Lr: 1.5404
Epoch 18/40, Training accuracy: 77.33%, Lr: 1.4634
Epoch 19/40, Training accuracy: 78.65%, Lr: 1.3903
Epoch 20/40, Training accuracy: 80.33%, 

In [ ]:
pred_percept = percept_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_fashion)))

The training accuracy is given by: 83.282000


### Validate Perceptron on Fashion-MNIST

In [ ]:
pred_percept = percept_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_fashion)))

The validation accuracy is given by: 82.380000


### Test Perceptron on Fashion-MNIST

In [ ]:
pred_percept = percept_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_fashion)))

The testing accuracy is given by: 81.590000


## Train Perceptron on Rice

In [ ]:
lr = 0.05
n_epochs = 15
n_features_RICE = X_train_RICE.shape[1]

percept_RICE = Perceptron(n_features_RICE, n_class_RICE, lr, n_epochs)
percept_RICE.train(X_train_RICE, y_train_RICE)

Epoch 1/15, Training accuracy: 53.48%, Lr: 0.0500
Epoch 2/15, Training accuracy: 54.78%, Lr: 0.0475
Epoch 3/15, Training accuracy: 45.22%, Lr: 0.0451
Epoch 4/15, Training accuracy: 54.78%, Lr: 0.0429
Epoch 5/15, Training accuracy: 45.24%, Lr: 0.0407
Epoch 6/15, Training accuracy: 54.78%, Lr: 0.0387
Epoch 7/15, Training accuracy: 48.01%, Lr: 0.0368
Epoch 8/15, Training accuracy: 54.78%, Lr: 0.0349
Epoch 9/15, Training accuracy: 56.37%, Lr: 0.0332
Epoch 10/15, Training accuracy: 54.88%, Lr: 0.0315
Epoch 11/15, Training accuracy: 50.67%, Lr: 0.0299
Epoch 12/15, Training accuracy: 90.24%, Lr: 0.0284
Epoch 13/15, Training accuracy: 99.66%, Lr: 0.0270
Epoch 14/15, Training accuracy: 99.69%, Lr: 0.0257
Epoch 15/15, Training accuracy: 99.74%, Lr: 0.0244


In [ ]:
pred_percept = percept_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_RICE)))

The training accuracy is given by: 99.761708


### Validate Perceptron on Rice

In [ ]:
pred_percept = percept_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_RICE)))

The validation accuracy is given by: 99.670058


### Test Perceptron on Rice

In [ ]:
pred_percept = percept_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_RICE)))

The testing accuracy is given by: 99.752543


# Support Vector Machines (with SGD)

In the next phase of the project, I implemented a soft-margin Support Vector Machine (SVM). The objective was to maximize the margin between positive and negative training samples, while applying a penalty for margin violations using a hinge loss.

To optimize the SVM loss, I used Stochastic Gradient Descent (SGD). This required computing the gradient of the loss with respect to the model’s weights and updating them iteratively.

The SVM model had three key hyperparameters that I tuned:
* **Learning Rate**: Determines the step size during gradient updates. Similar to the Perceptron, I experimented with different values to observe the impact on convergence and model performance.
* **Epochs**: Specifies the number of times the entire dataset is passed through during training. I adjusted this based on how quickly the model was converging.
* **Regularization Constant**: Controls the strength of regularization, influencing how strictly the model maximizes the margin. I tested different values, starting with a default of 0.05, to see how it affected the trade-off between margin size and misclassification.

The SVM implementation was developed in **models/svm.py**. Once the class was defined, I:
* Created an instance of the SVM classifier,
* Trained it on the dataset using the .train() method,
* Evaluated both training and testing accuracy using the .predict() method.

This implementation allowed me to directly compare the performance of the SVM model with the previously built Perceptron, providing deeper insight into the behavior of linear classifiers under different optimization strategies.

## Train SVM on Fashion-MNIST

In [ ]:
# Adjust the following four hyperparameters to achieve the best accuracy:
lr = 0.5
n_epochs = 40
reg_const = 0.05
batch_size = 256

svm_fashion = SVM(n_features_fashion, n_class_fashion, lr, n_epochs, reg_const)

# Refer to hints in the calculate_gradient and train methods of the SVM class in "models/svm.py"
svm_fashion.train(X_train_fashion, y_train_fashion, batch_size)

Epoch 1/40, Training accuracy: 75.68%, Lr: 0.5000
Epoch 2/40, Training accuracy: 77.96%, Lr: 0.4750
Epoch 3/40, Training accuracy: 75.28%, Lr: 0.4512
Epoch 4/40, Training accuracy: 77.37%, Lr: 0.4287
Epoch 5/40, Training accuracy: 76.21%, Lr: 0.4073
Epoch 6/40, Training accuracy: 66.68%, Lr: 0.3869
Epoch 7/40, Training accuracy: 81.16%, Lr: 0.3675
Epoch 8/40, Training accuracy: 81.14%, Lr: 0.3492
Epoch 9/40, Training accuracy: 81.79%, Lr: 0.3317
Epoch 10/40, Training accuracy: 82.09%, Lr: 0.3151
Epoch 11/40, Training accuracy: 79.68%, Lr: 0.2994
Epoch 12/40, Training accuracy: 78.82%, Lr: 0.2844
Epoch 13/40, Training accuracy: 83.51%, Lr: 0.2702
Epoch 14/40, Training accuracy: 83.04%, Lr: 0.2567
Epoch 15/40, Training accuracy: 81.95%, Lr: 0.2438
Epoch 16/40, Training accuracy: 83.81%, Lr: 0.2316
Epoch 17/40, Training accuracy: 82.72%, Lr: 0.2201
Epoch 18/40, Training accuracy: 74.37%, Lr: 0.2091
Epoch 19/40, Training accuracy: 84.35%, Lr: 0.1986
Epoch 20/40, Training accuracy: 80.46%, 

In [ ]:
pred_svm = svm_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_fashion)))

The training accuracy is given by: 85.424000


### Validate SVM on Fashion-MNIST

In [ ]:
pred_svm = svm_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_fashion)))

The validation accuracy is given by: 81.690000


### Test SVM on Fashion-MNIST

In [ ]:
pred_svm = svm_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_fashion)))

The testing accuracy is given by: 81.350000


## Train SVM on Rice

In [ ]:
# Adjust the following four hyperparameters to achieve the best accuracy:
lr = 0.05
n_epochs = 5
reg_const = 0.05
batch_size = 256

svm_RICE = SVM(n_features_RICE, n_class_RICE, lr, n_epochs, reg_const)
svm_RICE.train(X_train_RICE, y_train_RICE, batch_size)

Epoch 1/5, Training accuracy: 99.97%, Lr: 0.0500
Epoch 2/5, Training accuracy: 99.97%, Lr: 0.0475
Epoch 3/5, Training accuracy: 99.95%, Lr: 0.0451
Epoch 4/5, Training accuracy: 99.96%, Lr: 0.0429
Epoch 5/5, Training accuracy: 99.97%, Lr: 0.0407


In [ ]:
pred_svm = svm_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_RICE)))

The training accuracy is given by: 99.972505


### Validate SVM on Rice

In [ ]:
pred_svm = svm_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_RICE)))

The validation accuracy is given by: 99.890019


## Test SVM on Rice

In [ ]:
pred_svm = svm_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_RICE)))

The testing accuracy is given by: 99.972505


# Softmax Classifier (with SGD)

As the final part of the project, I implemented a Softmax classifier. This model applies a linear transformation to the input data, followed by a softmax function that outputs a probability distribution over multiple classes. Each element of the output vector represents the model’s confidence in one of the possible classes, with all values summing to 1. The model was trained using a cross-entropy loss.

To optimize the loss, I used Stochastic Gradient Descent (SGD). This required computing the gradient of the softmax cross-entropy loss with respect to the model weights and updating the weights accordingly.

The softmax classifier included the following tunable hyperparameters:
* **Learning Rate** – Determines the step size of weight updates based on the computed gradients.
* **Number of Epochs** – Sets how many complete passes are made over the training dataset.
* **Regularization Constant** – Controls the strength of L2 regularization applied to the weights to prevent overfitting. I experimented with different values to find a good balance between model complexity and generalization.

The implementation was done in **models/softmax.py**. After defining the class:
* I instantiated the Softmax classifier.
* Trained it on the dataset using the .train() method.
* Evaluated its performance by measuring training and testing accuracy with the .predict() method.

This completed the set of linear models explored in this project, giving me a broader understanding of how different loss functions and optimization approaches influence classification performance.

## Train Softmax on Fashion-MNIST

In [ ]:
lr = 0.5
n_epochs = 15
reg_const = 0.5

softmax_fashion = Softmax(n_class_fashion, lr, n_epochs, reg_const)

# Refer to hints in the calculate_gradient and train methods of the Softmax class in "models/softmax.py"
softmax_fashion.train(X_train_fashion, y_train_fashion)

Epoch 1: training accuracy = 0.6318
Epoch 2: training accuracy = 0.6660
Epoch 3: training accuracy = 0.6281
Epoch 4: training accuracy = 0.7341
Epoch 5: training accuracy = 0.8246
Epoch 6: training accuracy = 0.8533
Epoch 7: training accuracy = 0.8536
Epoch 8: training accuracy = 0.8552
Epoch 9: training accuracy = 0.8553
Epoch 10: training accuracy = 0.8552
Epoch 11: training accuracy = 0.8552
Epoch 12: training accuracy = 0.8552
Epoch 13: training accuracy = 0.8552
Epoch 14: training accuracy = 0.8552
Epoch 15: training accuracy = 0.8552


In [ ]:
pred_softmax = softmax_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_fashion)))

The training accuracy is given by: 85.518000


### Validate Softmax on Fashion-MNIST

In [ ]:
pred_softmax = softmax_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_fashion)))

The validation accuracy is given by: 83.950000


### Testing Softmax on Fashion-MNIST

In [ ]:
pred_softmax = softmax_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_fashion)))

The testing accuracy is given by: 83.160000


## Train Softmax on Rice

In [ ]:
lr = 0.4
n_epochs = 5
reg_const = 0.05

softmax_RICE = Softmax(n_class_RICE, lr, n_epochs, reg_const)
softmax_RICE.train(X_train_RICE, y_train_RICE)

Epoch 1: training accuracy = 0.9995
Epoch 2: training accuracy = 1.0000
Epoch 3: training accuracy = 1.0000
Epoch 4: training accuracy = 1.0000
Epoch 5: training accuracy = 1.0000


In [ ]:
pred_softmax = softmax_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_RICE)))

The training accuracy is given by: 100.000000


### Validate Softmax on Rice

In [ ]:
pred_softmax = softmax_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_RICE)))

The validation accuracy is given by: 99.945010


### Testing Softmax on Rice

In [ ]:
pred_softmax = softmax_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_RICE)))

The testing accuracy is given by: 100.000000


# Logistic Classifier

I also implemented a Logistic Regression classifier to explore binary classification using probabilistic decision boundaries. The model outputs values between 0 and 1, and a threshold is applied to determine the final class label.

The implementation included the following key hyperparameters:
* **Learning Rate** – Controls the step size for updating weights based on the gradient.
* **Number of Epochs** – Determines how many times the full training dataset is passed through during training.
* **Threshold** – Sets the decision boundary for classification. Predictions above this threshold are assigned to one class, and those below to the other.

The classifier was implemented in **models/logistic.py**. After defining the class, I:
* Created an instance of the logistic regression model.
* Trained it on the dataset using the .train() method.
* Evaluated its performance using the .predict() method to calculate both training and testing accuracy.

This model provided a useful comparison point to the Perceptron, SVM, and Softmax classifiers, especially in terms of interpretability and performance on linearly separable data.

### Training Logistic Classifer

In [ ]:
learning_rate = 0.5
n_epochs = 10
threshold = 0.5

lr = Logistic(learning_rate, n_epochs, threshold)

# Refer to hints in the sigmoid and train methods of the Logistic class in "models/logistic.py"
lr.train(X_train_RICE, y_train_RICE)

array([  1.95426249,   5.04570194, -17.28536458,  -4.18977314,
         4.77498026,  -8.99961048,   9.64371841,  -5.1267166 ,
        -4.11631723,  -0.33664439,  -0.08039806,  -2.80055505,
        11.42967033])

In [ ]:
pred_lr = lr.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_lr, y_train_RICE)))

The training accuracy is given by: 100.000000


### Validate Logistic Classifer

In [ ]:
pred_lr = lr.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_lr, y_val_RICE)))

The validation accuracy is given by: 99.945010


### Test Logistic Classifier

In [ ]:
pred_lr = lr.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_lr, y_test_RICE)))

The testing accuracy is given by: 100.000000
